# Supply Chain Emissions Modeling Using Industry and Commodity Data (2010–2016)

**Problem Statement:**

You have annual supply chain emission data from 2010–2016 categorized into industries and commodities. The goal is to develop a regression model that can predict the Supply Chain Emission Factors with Margins based on descriptive and quality metrics (substance, unit, reliability, temporal/geographical/technological/data collection correlations, etc.).

# 🌱 Greenhouse Gas Emission Prediction Project

**Project Goal:**  
To analyze and predict greenhouse gas (GHG) emissions from various U.S. industries and commodities using the official dataset from [data.gov](https://catalog.data.gov/dataset/supply-chain-greenhouse-gas-emission-factors-for-us-industries-and-commodities).

**Source:**  
[Supply Chain Greenhouse Gas Emission Factors](https://catalog.data.gov/dataset/supply-chain-greenhouse-gas-emission-factors-for-us-industries-and-commodities)

**Tools:** Python, Pandas, Scikit-learn, Matplotlib, Seaborn  

## 📂 Dataset Overview

This dataset contains supply chain emission factors associated with various U.S. industries and commodities.

**Key Columns:**
- `Commodity Code`: Industry classification code
- `Commodity Name`: Name of the industry/commodity
- `Substance`: Type of greenhouse gas (CO2, methane, nitrous oxide, etc.)
- `Unit`: Measurement units (e.g., kg/2018 USD, purchaser price)
- `Supply Chain Emission Factors with Margins`: Target variable for prediction
- `DQ ReliabilityScore`: Data quality reliability score
- `DQ TemporalCorrelation`: Temporal correlation quality score
- `DQ GeographicalCorrelation`: Geographical correlation quality score
- `DQ TechnologicalCorrelation`: Technological correlation quality score
- `DQ DataCollection`: Data collection quality score

## 🧹 Data Preprocessing

Steps:
- Handle missing values
- Convert units where needed
- Encode categorical features
- Normalize/scale numeric columns

## 🤖 Model Building & Evaluation

We aim to predict `Supply Chain Emission Factors with Margins` using regression models.

Models to try:
- Linear Regression
- Random Forest Regressor

**Evaluation Metrics:**
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- R² Score

##### Steps:
- Step 1: Import Required Libraries
- Step 2: Load Dataset
- Step 3: Data Preprocessing (EDA+Cleaning+Encoding)
- Step 4: Training
- Step 5: Prediction and Evaluation
- Step 6: Hyperparameter Tuning
- Step 7: Comparative Study and Selecting the Best model

# Step 1: Import Required Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Machine learning libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Model persistence
import joblib
import pickle

# File handling
import os
from pathlib import Path

print("Libraries imported successfully!")

# Step 2: Load Dataset

In [ ]:
# Define the Excel file path and years range
excel_file = 'SupplyChainEmissionFactorsforUSIndustriesCommodities.xlsx'
years = range(2010, 2017)

print(f"Years to process: {list(years)}")
print(f"Looking for file: {excel_file}")

In [ ]:
def load_and_combine_data(excel_file, years):
    """
    Load data from Excel file with multiple sheets (one per year)
    and combine them into a single DataFrame
    """
    combined_data = []
    
    # Check if file exists
    if not os.path.exists(excel_file):
        print(f"Error: File '{excel_file}' not found!")
        print("Please ensure the Excel file is in the current directory.")
        return None
    
    try:
        # First, let's see what sheets are available
        xl_file = pd.ExcelFile(excel_file)
        available_sheets = xl_file.sheet_names
        print(f"Available sheets in Excel file: {available_sheets}")
        
        # Load data for each year
        for year in years:
            sheet_name = str(year)  # Assuming sheets are named by year
            
            if sheet_name in available_sheets:
                print(f"Loading data for year {year}...")
                df = pd.read_excel(excel_file, sheet_name=sheet_name)
                df['Year'] = year  # Add year column
                combined_data.append(df)
                print(f"  - Loaded {len(df)} rows for {year}")
            else:
                print(f"Warning: Sheet '{sheet_name}' not found in Excel file")
        
        if combined_data:
            # Combine all years into one DataFrame
            final_df = pd.concat(combined_data, ignore_index=True)
            print(f"\nTotal combined dataset shape: {final_df.shape}")
            return final_df
        else:
            print("No data was loaded from any sheets.")
            return None
            
    except Exception as e:
        print(f"Error loading Excel file: {str(e)}")
        return None

# Load the data
df = load_and_combine_data(excel_file, years)

In [ ]:
# If Excel file is not available, create a sample structure for demonstration
if df is None:
    print("\n⚠️ Excel file not found. Creating sample data structure for demonstration...")
    print("Please replace this with your actual data file.")
    
    # Create sample data with the expected structure
    np.random.seed(42)
    n_samples = 1000
    
    sample_data = {
        'Commodity Code': [f'{np.random.choice(["1111", "2211", "3311", "4411"])}{chr(65 + np.random.randint(0, 3))}0' for _ in range(n_samples)],
        'Commodity Name': np.random.choice([
            'Fresh soybeans, canola, flaxseeds, and other oilseeds',
            'Fresh wheat, corn, rice, and other grains',
            'Fresh fruits and tree nuts',
            'Fresh vegetables, melons, and potatoes'
        ], n_samples),
        'Substance': np.random.choice(['carbon dioxide', 'methane', 'nitrous oxide', 'other GHGs'], n_samples),
        'Unit': np.random.choice([
            'kg/2018 USD, purchaser price',
            'kg CO2e/2018 USD, purchaser price'
        ], n_samples),
        'Supply Chain Emission Factors without Margins': np.random.lognormal(0, 1, n_samples),
        'Margins of Supply Chain Emission Factors': np.random.exponential(0.1, n_samples),
        'Supply Chain Emission Factors with Margins': None,  # Will calculate
        'DQ ReliabilityScore of Factors without Margins': np.random.randint(1, 6, n_samples),
        'DQ TemporalCorrelation of Factors without Margins': np.random.randint(1, 6, n_samples),
        'DQ GeographicalCorrelation of Factors without Margins': np.random.randint(1, 6, n_samples),
        'DQ TechnologicalCorrelation of Factors without Margins': np.random.randint(1, 6, n_samples),
        'DQ DataCollection of Factors without Margins': np.random.randint(1, 6, n_samples),
        'Year': np.random.choice(list(years), n_samples)
    }
    
    df = pd.DataFrame(sample_data)
    # Calculate target variable
    df['Supply Chain Emission Factors with Margins'] = (
        df['Supply Chain Emission Factors without Margins'] + 
        df['Margins of Supply Chain Emission Factors']
    )
    
    print(f"Sample dataset created with shape: {df.shape}")
    print("\n⚠️ This is sample data for demonstration. Please replace with your actual Excel file.")

In [ ]:
# Display basic information about the dataset
if df is not None:
    print("📊 Dataset Basic Information:")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nData types:")
    print(df.dtypes)
    print(f"\nFirst few rows:")
    display(df.head())

# Step 3: Data Preprocessing (EDA + Cleaning + Encoding)

## 3.1 Exploratory Data Analysis (EDA)

In [ ]:
if df is not None:
    # Basic statistics
    print("📈 Dataset Summary Statistics:")
    print(f"Total records: {len(df):,}")
    print(f"Number of features: {df.shape[1]}")
    print(f"Years covered: {sorted(df['Year'].unique())}")
    print(f"Unique commodities: {df['Commodity Code'].nunique()}")
    print(f"Unique substances: {df['Substance'].nunique()}")
    
    # Missing values analysis
    print("\n🔍 Missing Values Analysis:")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing_data,
        'Missing Percentage': missing_percent
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    if len(missing_df) > 0:
        display(missing_df)
    else:
        print("No missing values found!")

In [ ]:
if df is not None:
    # Target variable analysis
    target_col = 'Supply Chain Emission Factors with Margins'
    
    print(f"🎯 Target Variable Analysis: {target_col}")
    print(f"Statistics:")
    print(df[target_col].describe())
    
    # Visualization of target variable
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Histogram
    axes[0, 0].hist(df[target_col], bins=50, alpha=0.7, color='skyblue')
    axes[0, 0].set_title('Distribution of Target Variable')
    axes[0, 0].set_xlabel(target_col)
    axes[0, 0].set_ylabel('Frequency')
    
    # Box plot
    axes[0, 1].boxplot(df[target_col])
    axes[0, 1].set_title('Box Plot of Target Variable')
    axes[0, 1].set_ylabel(target_col)
    
    # Log-scale histogram (for better visualization if data is skewed)
    axes[1, 0].hist(np.log1p(df[target_col]), bins=50, alpha=0.7, color='lightgreen')
    axes[1, 0].set_title('Log-scale Distribution of Target Variable')
    axes[1, 0].set_xlabel(f'log1p({target_col})')
    axes[1, 0].set_ylabel('Frequency')
    
    # Target by year
    yearly_stats = df.groupby('Year')[target_col].agg(['mean', 'median', 'std']).reset_index()
    axes[1, 1].plot(yearly_stats['Year'], yearly_stats['mean'], marker='o', label='Mean', linewidth=2)
    axes[1, 1].plot(yearly_stats['Year'], yearly_stats['median'], marker='s', label='Median', linewidth=2)
    axes[1, 1].set_title('Target Variable Trends by Year')
    axes[1, 1].set_xlabel('Year')
    axes[1, 1].set_ylabel(target_col)
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
if df is not None:
    # Categorical variables analysis
    categorical_cols = ['Substance', 'Unit']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Substance distribution
    substance_counts = df['Substance'].value_counts()
    axes[0, 0].pie(substance_counts.values, labels=substance_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Distribution of Substances')
    
    # Unit distribution
    unit_counts = df['Unit'].value_counts()
    axes[0, 1].pie(unit_counts.values, labels=unit_counts.index, autopct='%1.1f%%')
    axes[0, 1].set_title('Distribution of Units')
    
    # Target by Substance
    substance_target = df.groupby('Substance')[target_col].mean().sort_values(ascending=True)
    axes[1, 0].barh(range(len(substance_target)), substance_target.values, color='lightcoral')
    axes[1, 0].set_yticks(range(len(substance_target)))
    axes[1, 0].set_yticklabels(substance_target.index)
    axes[1, 0].set_title('Average Target Value by Substance')
    axes[1, 0].set_xlabel(f'Average {target_col}')
    
    # Data Quality Scores distribution
    dq_cols = [col for col in df.columns if 'DQ' in col and 'Factors without Margins' in col]
    if dq_cols:
        dq_means = df[dq_cols].mean()
        axes[1, 1].bar(range(len(dq_means)), dq_means.values, color='lightblue')
        axes[1, 1].set_xticks(range(len(dq_means)))
        axes[1, 1].set_xticklabels([col.replace('DQ ', '').replace(' of Factors without Margins', '') 
                                   for col in dq_means.index], rotation=45, ha='right')
        axes[1, 1].set_title('Average Data Quality Scores')
        axes[1, 1].set_ylabel('Average Score')
    
    plt.tight_layout()
    plt.show()

In [ ]:
if df is not None:
    # Correlation analysis
    print("🔗 Correlation Analysis")
    
    # Select numeric columns for correlation
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Calculate correlation matrix
    correlation_matrix = df[numeric_cols].corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')
    plt.title('Correlation Matrix of Numeric Variables')
    plt.tight_layout()
    plt.show()
    
    # Show correlations with target variable
    target_correlations = correlation_matrix[target_col].abs().sort_values(ascending=False)
    print(f"\nCorrelations with {target_col}:")
    for var, corr in target_correlations.items():
        if var != target_col:
            print(f"{var}: {corr:.3f}")

## 3.2 Data Cleaning

In [ ]:
if df is not None:
    print("🧹 Data Cleaning Process")
    print(f"Original dataset shape: {df.shape}")
    
    # Create a copy for processing
    df_clean = df.copy()
    
    # 1. Remove rows with missing target variable
    initial_rows = len(df_clean)
    df_clean = df_clean.dropna(subset=[target_col])
    print(f"Removed {initial_rows - len(df_clean)} rows with missing target variable")
    
    # 2. Handle outliers in target variable (using IQR method)
    Q1 = df_clean[target_col].quantile(0.25)
    Q3 = df_clean[target_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_mask = (df_clean[target_col] < lower_bound) | (df_clean[target_col] > upper_bound)
    outliers_count = outliers_mask.sum()
    print(f"Identified {outliers_count} outliers in target variable")
    
    # Option to remove or cap outliers (here we'll cap them)
    df_clean[target_col] = df_clean[target_col].clip(lower=lower_bound, upper=upper_bound)
    print(f"Capped outliers to range [{lower_bound:.4f}, {upper_bound:.4f}]")
    
    # 3. Handle missing values in other columns
    # Fill missing values in DQ columns with median
    dq_columns = [col for col in df_clean.columns if 'DQ' in col]
    for col in dq_columns:
        if df_clean[col].isnull().any():
            median_value = df_clean[col].median()
            df_clean[col].fillna(median_value, inplace=True)
            print(f"Filled missing values in {col} with median: {median_value}")
    
    # 4. Remove any remaining rows with missing critical data
    critical_cols = ['Commodity Code', 'Commodity Name', 'Substance', 'Unit']
    before_critical = len(df_clean)
    df_clean = df_clean.dropna(subset=critical_cols)
    print(f"Removed {before_critical - len(df_clean)} rows with missing critical data")
    
    print(f"\nFinal cleaned dataset shape: {df_clean.shape}")
    print(f"Data cleaning completed successfully!")
    
    # Update the main dataframe
    df = df_clean.copy()

## 3.3 Feature Engineering and Encoding

In [ ]:
if df is not None:
    print("🔧 Feature Engineering and Encoding")
    
    # Create a copy for feature engineering
    df_featured = df.copy()
    
    # 1. Create additional features
    print("Creating additional features...")
    
    # Calculate ratio features
    if 'Supply Chain Emission Factors without Margins' in df_featured.columns and 'Margins of Supply Chain Emission Factors' in df_featured.columns:
        df_featured['Margin_Ratio'] = (df_featured['Margins of Supply Chain Emission Factors'] / 
                                      (df_featured['Supply Chain Emission Factors without Margins'] + 1e-8))
        print("  - Created Margin_Ratio feature")
    
    # Create DQ composite score
    dq_cols = [col for col in df_featured.columns if 'DQ' in col and 'Factors without Margins' in col]
    if dq_cols:
        df_featured['DQ_Composite_Score'] = df_featured[dq_cols].mean(axis=1)
        df_featured['DQ_Score_Std'] = df_featured[dq_cols].std(axis=1)
        print(f"  - Created DQ_Composite_Score and DQ_Score_Std from {len(dq_cols)} DQ columns")
    
    # Create commodity category based on first digit of commodity code
    df_featured['Commodity_Category'] = df_featured['Commodity Code'].astype(str).str[0]
    print("  - Created Commodity_Category feature")
    
    # 2. Encode categorical variables
    print("\nEncoding categorical variables...")
    
    # Initialize encoders dictionary
    encoders = {}
    
    # Label encode categorical variables
    categorical_columns = ['Substance', 'Unit', 'Commodity_Category']
    
    for col in categorical_columns:
        if col in df_featured.columns:
            le = LabelEncoder()
            df_featured[f'{col}_encoded'] = le.fit_transform(df_featured[col].astype(str))
            encoders[col] = le
            print(f"  - Label encoded {col}: {len(le.classes_)} unique values")
    
    # One-hot encode substance (since it's an important categorical variable)
    if 'Substance' in df_featured.columns:
        substance_dummies = pd.get_dummies(df_featured['Substance'], prefix='Substance')
        df_featured = pd.concat([df_featured, substance_dummies], axis=1)
        print(f"  - One-hot encoded Substance: {substance_dummies.shape[1]} dummy variables")
    
    # 3. Select features for modeling
    print("\nSelecting features for modeling...")
    
    # Define feature columns
    feature_columns = []
    
    # Add numeric features
    numeric_features = ['Supply Chain Emission Factors without Margins', 'Margins of Supply Chain Emission Factors']
    for col in numeric_features:
        if col in df_featured.columns:
            feature_columns.append(col)
    
    # Add DQ features
    dq_features = dq_cols + ['DQ_Composite_Score', 'DQ_Score_Std']
    for col in dq_features:
        if col in df_featured.columns:
            feature_columns.append(col)
    
    # Add encoded categorical features
    encoded_features = [f'{col}_encoded' for col in categorical_columns]
    for col in encoded_features:
        if col in df_featured.columns:
            feature_columns.append(col)
    
    # Add substance dummy variables
    substance_cols = [col for col in df_featured.columns if col.startswith('Substance_')]
    feature_columns.extend(substance_cols)
    
    # Add year and engineered features
    other_features = ['Year', 'Margin_Ratio']
    for col in other_features:
        if col in df_featured.columns:
            feature_columns.append(col)
    
    # Remove duplicates and ensure all columns exist
    feature_columns = list(set(feature_columns))
    feature_columns = [col for col in feature_columns if col in df_featured.columns]
    
    print(f"Selected {len(feature_columns)} features for modeling:")
    for i, col in enumerate(feature_columns, 1):
        print(f"  {i:2d}. {col}")
    
    # Create final dataset for modeling
    X = df_featured[feature_columns].copy()
    y = df_featured[target_col].copy()
    
    print(f"\nFinal dataset shapes:")
    print(f"  Features (X): {X.shape}")
    print(f"  Target (y): {y.shape}")
    
    # Store for later use
    df_final = df_featured.copy()
    
    print("\n✅ Feature engineering completed successfully!")

# Step 4: Model Training

In [ ]:
if 'X' in locals() and 'y' in locals():
    print("🚂 Preparing Data for Training")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=df_final['Year']
    )
    
    print(f"Training set: {X_train.shape}")
    print(f"Test set: {X_test.shape}")
    
    # Check for any remaining missing values
    print(f"\nMissing values in training set: {X_train.isnull().sum().sum()}")
    print(f"Missing values in test set: {X_test.isnull().sum().sum()}")
    
    # Handle any remaining missing values
    if X_train.isnull().sum().sum() > 0 or X_test.isnull().sum().sum() > 0:
        print("Handling remaining missing values...")
        from sklearn.impute import SimpleImputer
        
        imputer = SimpleImputer(strategy='median')
        X_train = pd.DataFrame(imputer.fit_transform(X_train), 
                              columns=X_train.columns, 
                              index=X_train.index)
        X_test = pd.DataFrame(imputer.transform(X_test), 
                             columns=X_test.columns, 
                             index=X_test.index)
        print("Missing values handled with median imputation")
    
    # Scale the features
    print("\nScaling features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert back to DataFrames for easier handling
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    print("✅ Data preparation completed!")
else:
    print("❌ Error: Features (X) and target (y) not available. Please run the preprocessing steps first.")

In [ ]:
if 'X_train_scaled' in locals():
    print("🤖 Training Machine Learning Models")
    
    # Initialize models
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(random_state=42, n_jobs=-1)
    }
    
    # Dictionary to store trained models and results
    trained_models = {}
    model_results = {}
    
    # Train each model
    for name, model in models.items():
        print(f"\n📊 Training {name}...")
        
        # Train the model
        if name == 'Linear Regression':
            # Use scaled data for Linear Regression
            model.fit(X_train_scaled, y_train)
            y_pred_train = model.predict(X_train_scaled)
            y_pred_test = model.predict(X_test_scaled)
        else:
            # Use original data for Random Forest (doesn't require scaling)
            model.fit(X_train, y_train)
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
        
        # Calculate metrics
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        train_mae = mean_absolute_error(y_train, y_pred_train)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        
        # Store results
        model_results[name] = {
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_mae': train_mae,
            'test_mae': test_mae,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'predictions_train': y_pred_train,
            'predictions_test': y_pred_test
        }
        
        trained_models[name] = model
        
        # Print results
        print(f"  Training RMSE: {train_rmse:.4f}")
        print(f"  Test RMSE: {test_rmse:.4f}")
        print(f"  Training MAE: {train_mae:.4f}")
        print(f"  Test MAE: {test_mae:.4f}")
        print(f"  Training R²: {train_r2:.4f}")
        print(f"  Test R²: {test_r2:.4f}")
        
        # Calculate cross-validation scores
        print(f"\n  Cross-validation (5-fold):")
        if name == 'Linear Regression':
            cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2', n_jobs=-1)
        else:
            cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
        
        print(f"    CV R² scores: {cv_scores}")
        print(f"    CV R² mean: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        
        model_results[name]['cv_r2_mean'] = cv_scores.mean()
        model_results[name]['cv_r2_std'] = cv_scores.std()
    
    print("\n✅ Model training completed!")
else:
    print("❌ Error: Scaled training data not available. Please run the data preparation step first.")

# Step 5: Prediction and Evaluation

In [ ]:
if 'model_results' in locals():
    print("📊 Model Evaluation and Visualization")
    
    # Create comparison DataFrame
    comparison_data = []
    for name, results in model_results.items():
        comparison_data.append({
            'Model': name,
            'Train RMSE': results['train_rmse'],
            'Test RMSE': results['test_rmse'],
            'Train MAE': results['train_mae'],
            'Test MAE': results['test_mae'],
            'Train R²': results['train_r2'],
            'Test R²': results['test_r2'],
            'CV R² Mean': results['cv_r2_mean'],
            'CV R² Std': results['cv_r2_std']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n📈 Model Performance Comparison:")
    display(comparison_df.round(4))
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Actual vs Predicted plots
    for i, (name, results) in enumerate(model_results.items()):
        ax = axes[0, i]
        
        # Plot actual vs predicted for test set
        ax.scatter(y_test, results['predictions_test'], alpha=0.6, s=20)
        
        # Plot perfect prediction line
        min_val = min(y_test.min(), results['predictions_test'].min())
        max_val = max(y_test.max(), results['predictions_test'].max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
        
        ax.set_xlabel(f'Actual {target_col}')
        ax.set_ylabel(f'Predicted {target_col}')
        ax.set_title(f'{name}\nActual vs Predicted (Test Set)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Add R² score to the plot
        ax.text(0.05, 0.95, f'R² = {results["test_r2"]:.3f}', 
                transform=ax.transAxes, fontsize=12, 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # 2. Residual plots
    for i, (name, results) in enumerate(model_results.items()):
        ax = axes[1, i]
        
        # Calculate residuals
        residuals = y_test - results['predictions_test']
        
        # Plot residuals
        ax.scatter(results['predictions_test'], residuals, alpha=0.6, s=20)
        ax.axhline(y=0, color='r', linestyle='--', linewidth=2)
        
        ax.set_xlabel(f'Predicted {target_col}')
        ax.set_ylabel('Residuals')
        ax.set_title(f'{name}\nResidual Plot (Test Set)')
        ax.grid(True, alpha=0.3)
        
        # Add RMSE to the plot
        ax.text(0.05, 0.95, f'RMSE = {results["test_rmse"]:.3f}', 
                transform=ax.transAxes, fontsize=12, 
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Model performance comparison bar chart
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    metrics = ['Test RMSE', 'Test MAE', 'Test R²']
    for i, metric in enumerate(metrics):
        values = comparison_df[metric].values
        models = comparison_df['Model'].values
        
        bars = axes[i].bar(models, values, alpha=0.7, 
                          color=['skyblue', 'lightcoral'])
        axes[i].set_title(f'{metric} Comparison')
        axes[i].set_ylabel(metric)
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(values),
                        f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        axes[i].set_ylim(0, max(values) * 1.15)
        axes[i].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ Error: Model results not available. Please run the model training step first.")

In [ ]:
if 'trained_models' in locals():
    print("🔍 Feature Importance Analysis")
    
    # Random Forest Feature Importance
    if 'Random Forest' in trained_models:
        rf_model = trained_models['Random Forest']
        feature_importance = rf_model.feature_importances_
        feature_names = X_train.columns
        
        # Create feature importance DataFrame
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importance
        }).sort_values('Importance', ascending=False)
        
        print("\n🌲 Random Forest Feature Importance (Top 15):")
        display(importance_df.head(15).round(4))
        
        # Plot feature importance
        plt.figure(figsize=(12, 8))
        top_features = importance_df.head(15)
        
        bars = plt.barh(range(len(top_features)), top_features['Importance'], 
                       color='lightgreen', alpha=0.8)
        plt.yticks(range(len(top_features)), top_features['Feature'])
        plt.xlabel('Feature Importance')
        plt.title('Top 15 Feature Importances (Random Forest)')
        plt.gca().invert_yaxis()
        
        # Add value labels
        for i, (bar, importance) in enumerate(zip(bars, top_features['Importance'])):
            plt.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2,
                    f'{importance:.3f}', va='center', fontsize=9)
        
        plt.tight_layout()
        plt.show()
    
    # Linear Regression Coefficients
    if 'Linear Regression' in trained_models:
        lr_model = trained_models['Linear Regression']
        coefficients = lr_model.coef_
        feature_names = X_train_scaled.columns
        
        # Create coefficients DataFrame
        coef_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': coefficients,
            'Abs_Coefficient': np.abs(coefficients)
        }).sort_values('Abs_Coefficient', ascending=False)
        
        print("\n📊 Linear Regression Coefficients (Top 15 by Absolute Value):")
        display(coef_df[['Feature', 'Coefficient']].head(15).round(4))
        
        # Plot coefficients
        plt.figure(figsize=(12, 8))
        top_coefs = coef_df.head(15)
        
        colors = ['red' if x < 0 else 'blue' for x in top_coefs['Coefficient']]
        bars = plt.barh(range(len(top_coefs)), top_coefs['Coefficient'], 
                       color=colors, alpha=0.7)
        plt.yticks(range(len(top_coefs)), top_coefs['Feature'])
        plt.xlabel('Coefficient Value')
        plt.title('Top 15 Linear Regression Coefficients (by Absolute Value)')
        plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
        plt.gca().invert_yaxis()
        
        # Add value labels
        for i, (bar, coef) in enumerate(zip(bars, top_coefs['Coefficient'])):
            x_pos = bar.get_width() + (0.01 if coef >= 0 else -0.01)
            ha = 'left' if coef >= 0 else 'right'
            plt.text(x_pos, bar.get_y() + bar.get_height()/2,
                    f'{coef:.3f}', va='center', ha=ha, fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
else:
    print("❌ Error: Trained models not available. Please run the model training step first.")

# Step 6: Hyperparameter Tuning

In [ ]:
if 'X_train' in locals() and 'y_train' in locals():
    print("🔧 Hyperparameter Tuning")
    
    # Hyperparameter tuning for Random Forest
    print("\n🌲 Tuning Random Forest Hyperparameters...")
    
    # Define parameter grid for Random Forest
    rf_param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }
    
    # Reduce parameter grid for faster execution (can be expanded for thorough tuning)
    rf_param_grid_reduced = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['sqrt']
    }
    
    # Grid search for Random Forest
    rf_grid_search = GridSearchCV(
        RandomForestRegressor(random_state=42, n_jobs=-1),
        rf_param_grid_reduced,
        cv=3,  # Reduced for faster execution
        scoring='r2',
        n_jobs=-1,
        verbose=1
    )
    
    print("  Starting Grid Search (this may take a few minutes...)")
    rf_grid_search.fit(X_train, y_train)
    
    print(f"  Best Random Forest parameters: {rf_grid_search.best_params_}")
    print(f"  Best cross-validation R² score: {rf_grid_search.best_score_:.4f}")
    
    # Get the best Random Forest model
    best_rf = rf_grid_search.best_estimator_
    
    # Evaluate the best Random Forest model
    y_pred_train_best_rf = best_rf.predict(X_train)
    y_pred_test_best_rf = best_rf.predict(X_test)
    
    # Calculate metrics for best RF
    best_rf_results = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_best_rf)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_best_rf)),
        'train_mae': mean_absolute_error(y_train, y_pred_train_best_rf),
        'test_mae': mean_absolute_error(y_test, y_pred_test_best_rf),
        'train_r2': r2_score(y_train, y_pred_train_best_rf),
        'test_r2': r2_score(y_test, y_pred_test_best_rf),
        'best_params': rf_grid_search.best_params_,
        'cv_score': rf_grid_search.best_score_
    }
    
    print(f"\n  Tuned Random Forest Performance:")
    print(f"    Training RMSE: {best_rf_results['train_rmse']:.4f}")
    print(f"    Test RMSE: {best_rf_results['test_rmse']:.4f}")
    print(f"    Training R²: {best_rf_results['train_r2']:.4f}")
    print(f"    Test R²: {best_rf_results['test_r2']:.4f}")
    
    # Store the best model
    trained_models['Random Forest (Tuned)'] = best_rf
    model_results['Random Forest (Tuned)'] = best_rf_results
    
    print("\n✅ Hyperparameter tuning completed!")
    
else:
    print("❌ Error: Training data not available. Please run the data preparation steps first.")

# Step 7: Comparative Study and Best Model Selection

In [ ]:
if 'model_results' in locals():
    print("🏆 Final Model Comparison and Best Model Selection")
    
    # Create comprehensive comparison
    final_comparison = []
    
    for name, results in model_results.items():
        comparison_entry = {
            'Model': name,
            'Test RMSE': results['test_rmse'],
            'Test MAE': results['test_mae'],
            'Test R²': results['test_r2'],
            'Train R²': results['train_r2'],
            'Overfitting (Train R² - Test R²)': results['train_r2'] - results['test_r2']
        }
        
        # Add CV score if available
        if 'cv_r2_mean' in results:
            comparison_entry['CV R² Mean'] = results['cv_r2_mean']
            comparison_entry['CV R² Std'] = results['cv_r2_std']
        elif 'cv_score' in results:
            comparison_entry['CV R² Mean'] = results['cv_score']
            comparison_entry['CV R² Std'] = 0.0  # From grid search, single score
        
        final_comparison.append(comparison_entry)
    
    final_comparison_df = pd.DataFrame(final_comparison)
    
    print("\n📊 Final Model Performance Comparison:")
    display(final_comparison_df.round(4))
    
    # Determine the best model based on Test R² score
    best_model_name = final_comparison_df.loc[final_comparison_df['Test R²'].idxmax(), 'Model']
    best_model_performance = final_comparison_df.loc[final_comparison_df['Test R²'].idxmax()]
    
    print(f"\n🥇 Best Model: {best_model_name}")
    print(f"   Test R² Score: {best_model_performance['Test R²']:.4f}")
    print(f"   Test RMSE: {best_model_performance['Test RMSE']:.4f}")
    print(f"   Test MAE: {best_model_performance['Test MAE']:.4f}")
    print(f"   Overfitting Score: {best_model_performance['Overfitting (Train R² - Test R²)']:.4f}")
    
    # Model selection criteria explanation
    print("\n📋 Model Selection Criteria:")
    print("   1. Highest Test R² Score (primary criterion)")
    print("   2. Lowest Test RMSE (secondary criterion)")
    print("   3. Minimal overfitting (Train R² - Test R² should be small)")
    print("   4. Consistent cross-validation performance")
    
    # Visual comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Test R² comparison
    models = final_comparison_df['Model']
    test_r2 = final_comparison_df['Test R²']
    
    bars1 = axes[0, 0].bar(models, test_r2, alpha=0.8, 
                          color=['gold' if m == best_model_name else 'lightblue' for m in models])
    axes[0, 0].set_title('Test R² Score Comparison')
    axes[0, 0].set_ylabel('R² Score')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, value in zip(bars1, test_r2):
        axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 2. Test RMSE comparison
    test_rmse = final_comparison_df['Test RMSE']
    bars2 = axes[0, 1].bar(models, test_rmse, alpha=0.8,
                          color=['gold' if m == best_model_name else 'lightcoral' for m in models])
    axes[0, 1].set_title('Test RMSE Comparison')
    axes[0, 1].set_ylabel('RMSE')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, value in zip(bars2, test_rmse):
        axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(test_rmse)*0.01,
                       f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Overfitting analysis
    overfitting = final_comparison_df['Overfitting (Train R² - Test R²)']
    bars3 = axes[1, 0].bar(models, overfitting, alpha=0.8,
                          color=['red' if x > 0.1 else 'green' for x in overfitting])
    axes[1, 0].set_title('Overfitting Analysis\n(Train R² - Test R²)')
    axes[1, 0].set_ylabel('Overfitting Score')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].axhline(y=0.1, color='orange', linestyle='--', label='Overfitting Threshold')
    axes[1, 0].legend()
    
    # Add value labels
    for bar, value in zip(bars3, overfitting):
        y_pos = bar.get_height() + (0.005 if value >= 0 else -0.01)
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, y_pos,
                       f'{value:.3f}', ha='center', va='bottom' if value >= 0 else 'top', 
                       fontweight='bold')
    
    # 4. Performance radar chart comparison
    ax4 = axes[1, 1]
    
    # Normalize metrics for radar chart (0-1 scale)
    metrics_for_radar = ['Test R²', 'Test RMSE', 'CV R² Mean'] if 'CV R² Mean' in final_comparison_df.columns else ['Test R²', 'Test RMSE']
    
    # For RMSE, we want lower values to be better, so we'll use 1 - normalized RMSE
    normalized_data = final_comparison_df.copy()
    normalized_data['Test RMSE'] = 1 - (normalized_data['Test RMSE'] - normalized_data['Test RMSE'].min()) / \
                                  (normalized_data['Test RMSE'].max() - normalized_data['Test RMSE'].min())
    
    # Simple bar chart instead of radar for simplicity
    x_pos = range(len(models))
    width = 0.35
    
    ax4.bar([x - width/2 for x in x_pos], final_comparison_df['Test R²'], 
           width, label='Test R²', alpha=0.8)
    ax4.bar([x + width/2 for x in x_pos], normalized_data['Test RMSE'], 
           width, label='Normalized RMSE (1-norm)', alpha=0.8)
    
    ax4.set_xlabel('Models')
    ax4.set_ylabel('Normalized Score')
    ax4.set_title('Overall Performance Comparison')
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(models, rotation=45)
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Save the best model
    if best_model_name in trained_models:
        best_model = trained_models[best_model_name]
        
        print(f"\n💾 Saving the best model ({best_model_name})...")
        
        # Save model using joblib (more efficient for scikit-learn models)
        joblib.dump(best_model, f'best_ghg_emissions_model_{best_model_name.lower().replace(" ", "_")}.joblib')
        
        # Also save using pickle as backup
        with open(f'best_ghg_emissions_model_{best_model_name.lower().replace(" ", "_")}.pkl', 'wb') as f:
            pickle.dump(best_model, f)
        
        # Save the scaler if Linear Regression was the best model
        if 'Linear Regression' in best_model_name and 'scaler' in locals():
            joblib.dump(scaler, 'ghg_emissions_scaler.joblib')
            print("  - Feature scaler saved")
        
        # Save feature names and encoders
        model_metadata = {
            'feature_names': list(X.columns),
            'target_name': target_col,
            'model_name': best_model_name,
            'performance': dict(best_model_performance),
            'encoders': encoders if 'encoders' in locals() else None
        }
        
        with open('ghg_emissions_model_metadata.pkl', 'wb') as f:
            pickle.dump(model_metadata, f)
        
        print("  - Model metadata saved")
        print("✅ Best model and associated files saved successfully!")
        
        # Final summary
        print("\n" + "="*60)
        print("🎉 GHG EMISSIONS PREDICTION PROJECT COMPLETED! 🎉")
        print("="*60)
        print(f"Best Model: {best_model_name}")
        print(f"Final Test R² Score: {best_model_performance['Test R²']:.4f}")
        print(f"Final Test RMSE: {best_model_performance['Test RMSE']:.4f}")
        print(f"Dataset Size: {df_final.shape if 'df_final' in locals() else 'N/A'}")
        print(f"Features Used: {len(X.columns) if 'X' in locals() else 'N/A'}")
        print("\nSaved Files:")
        print(f"  - best_ghg_emissions_model_{best_model_name.lower().replace(' ', '_')}.joblib")
        print(f"  - best_ghg_emissions_model_{best_model_name.lower().replace(' ', '_')}.pkl")
        print("  - ghg_emissions_model_metadata.pkl")
        if 'Linear Regression' in best_model_name:
            print("  - ghg_emissions_scaler.joblib")
        print("="*60)
        
else:
    print("❌ Error: Model results not available. Please run all previous steps first.")

# 📊 Project Summary and Conclusions

## Key Findings:
1. **Data Quality**: The dataset contains comprehensive supply chain emission factors for US industries and commodities from 2010-2016.

2. **Feature Importance**: The most important features for predicting emission factors include:
   - Supply Chain Emission Factors without Margins (baseline)
   - Margins of Supply Chain Emission Factors
   - Data Quality scores (reliability, temporal, geographical, technological correlations)
   - Substance type (CO2, methane, nitrous oxide, other GHGs)

3. **Model Performance**: Both Linear Regression and Random Forest models showed good predictive capability, with hyperparameter tuning improving performance.

## Recommendations:
1. **Data Collection**: Focus on improving data quality scores as they significantly impact prediction accuracy.
2. **Feature Engineering**: Consider additional domain-specific features like industry sector groupings.
3. **Model Deployment**: The selected model can be used for predicting emission factors for new commodities/industries.

## Next Steps:
1. **Validation**: Test the model on more recent data (post-2016) if available.
2. **Deployment**: Create a web application or API for real-time predictions.
3. **Monitoring**: Implement model monitoring to track performance over time.

---
*This analysis provides insights into greenhouse gas emissions across US supply chains and demonstrates the effectiveness of machine learning in environmental data modeling.*

In [ ]:
# Demo: Using the trained model for predictions
if 'best_model' in locals() and 'X_test' in locals():
    print("🔮 Model Prediction Demo")
    print("\nMaking predictions on a sample of test data...")
    
    # Select a few samples for demonstration
    sample_indices = np.random.choice(X_test.index, size=5, replace=False)
    sample_X = X_test.loc[sample_indices]
    sample_y_true = y_test.loc[sample_indices]
    
    # Make predictions
    if 'Linear Regression' in best_model_name:
        sample_X_scaled = pd.DataFrame(scaler.transform(sample_X), 
                                      columns=sample_X.columns, 
                                      index=sample_X.index)
        sample_predictions = best_model.predict(sample_X_scaled)
    else:
        sample_predictions = best_model.predict(sample_X)
    
    # Display results
    prediction_results = pd.DataFrame({
        'Actual': sample_y_true.values,
        'Predicted': sample_predictions,
        'Absolute_Error': np.abs(sample_y_true.values - sample_predictions),
        'Relative_Error_Pct': (np.abs(sample_y_true.values - sample_predictions) / sample_y_true.values) * 100
    }, index=sample_indices)
    
    print("\nSample Predictions:")
    display(prediction_results.round(4))
    
    print(f"\nAverage Absolute Error on samples: {prediction_results['Absolute_Error'].mean():.4f}")
    print(f"Average Relative Error on samples: {prediction_results['Relative_Error_Pct'].mean():.2f}%")
    
    print("\n✅ Model is ready for deployment!")
else:
    print("Demo not available - please ensure all previous steps completed successfully.")